In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import os

# Initialize variables to store overall accuracy results
average_accuracies = []

# Loop through n_neighbors values from 1 to 52
for n in range(5, 6):
    accuracies = []  # Store accuracy for each group

    # Loop through group numbers from 1 to 5
    for group_number in range(1, 6):
        train_all_path = f'../../data_preprocess/dataset/{group_number}/train_all.csv'
        test_path = f'../../translation/0.result/{group_number}/test_p.csv'

        if not os.path.exists(test_path):
            print(f"Test file for Group {group_number} does not exist. Skipping...")
            continue

        # Load the train_all and test CSVs
        train_all_csv = pd.read_csv(train_all_path, low_memory=False)
        test_csv = pd.read_csv(test_path, low_memory=False)

        train_all_csv['tag_description'] = train_all_csv['tag_description'].fillna('')
        test_csv['tag_description'] = test_csv['tag_description'].fillna('')

        test_csv['c_thing'] = ''
        test_csv['c_property'] = ''
        test_csv['c_score'] = ''
        test_csv['c_duplicate'] = 0

        combined_tag_descriptions = train_all_csv['tag_description'].tolist()

        # TfidfVectorizer 사용
        vectorizer = TfidfVectorizer(token_pattern=r'\S+', ngram_range=(1, 1), use_idf=True)
        vectorizer.fit(combined_tag_descriptions)

        train_all_tfidf_matrix = vectorizer.transform(train_all_csv['tag_description'])
        test_tfidf_matrix = vectorizer.transform(test_csv['tag_description'])

        # KNN에서 유클리디안 거리를 이용
        knn = NearestNeighbors(n_neighbors=n, metric='cosine', n_jobs=-1)
        knn.fit(train_all_tfidf_matrix)

        distances, indices = knn.kneighbors(test_tfidf_matrix)

        predicted_things = []
        predicted_properties = []
        predicted_scores = []

        for i in range(len(test_csv)):
            neighbor_index = indices[i][0]
            distance = distances[i][0]

            neighbor_thing = train_all_csv.iloc[neighbor_index]['thing']
            neighbor_property = train_all_csv.iloc[neighbor_index]['property']

            predicted_things.append(neighbor_thing)
            predicted_properties.append(neighbor_property)

            # 거리 기반으로 유사도 점수 계산
            predicted_score = 1 - distance
            predicted_scores.append(predicted_score)

        test_csv['c_thing'] = predicted_things
        test_csv['c_property'] = predicted_properties
        test_csv['c_score'] = predicted_scores

        test_csv['cthing_correct'] = test_csv['thing'] == test_csv['c_thing']
        test_csv['cproperty_correct'] = test_csv['property'] == test_csv['c_property']
        test_csv['ctp_correct'] = test_csv['cthing_correct'] & test_csv['cproperty_correct']

        mdm_true_count = len(test_csv[test_csv['MDM'] == True])
        accuracy = (test_csv['ctp_correct'].sum() / mdm_true_count) * 100
        accuracies.append(accuracy)

        # n_neighbors가 5일 때, test_csv를 지정된 경로에 저장
        if n == 5:
            output_path = f'0.class_document/knn_tfidf/{group_number}/test_p_c.csv'
            os.makedirs(os.path.dirname(output_path), exist_ok=True)  # 폴더가 없을 경우 생성
            test_csv.to_csv(output_path, index=False)
            print(f"test_p_c.csv saved for Group {group_number} at {output_path}")

    # Calculate the average accuracy for the current n_neighbors value
    average_accuracy = sum(accuracies) / len(accuracies)
    average_accuracies.append(average_accuracy)
    print(f"Average Accuracy (MDM=True) across all groups with n_neighbors={n}: {average_accuracy:.2f}%")

# Print overall results for all n_neighbors values
print("\nFinal Results:")
for n, avg_accuracy in zip(range(1, 53), average_accuracies):
    print(f"n_neighbors={n}, Average Accuracy: {avg_accuracy:.2f}%")


test_p_c.csv saved for Group 1 at 0.class_document/knn_tfidf/1/test_p_c.csv
test_p_c.csv saved for Group 2 at 0.class_document/knn_tfidf/2/test_p_c.csv
test_p_c.csv saved for Group 3 at 0.class_document/knn_tfidf/3/test_p_c.csv
test_p_c.csv saved for Group 4 at 0.class_document/knn_tfidf/4/test_p_c.csv
test_p_c.csv saved for Group 5 at 0.class_document/knn_tfidf/5/test_p_c.csv
Average Accuracy (MDM=True) across all groups with n_neighbors=5: 84.37%

Final Results:
n_neighbors=1, Average Accuracy: 84.37%
